# Statement
Una cámara de presión está sellada con una tapa sujeta con ocho pernos
pre-cargados. 

> Obtenga la rigidez del empaque y las constantes de la junta para dos diseños del ensamble, como se indica en la figura 11-33, uno con un empaque confinado y otro con un empaque no confinado. 

También determine las cargas experimentadas por los pernos y el material.

### Data
El diámetro del cilindro Dp = 4 in. El diámetro del círculo de los pernos
Dbc = 5.5 in. El diámetro de la pestaña exterior Df = 7.25 in. Los ocho
pernos 3/8-16 UNC están equidistantes en el círculo de pernos. La pestaña
sobre la cámara de acero tiene 0.75 in de grueso. El espesor de la cubierta
de aluminio lc = 1.125 in. La longitud sujeta l de la junta es de 1.875 in. El
espesor del empaque t = 0.125 in. La presión en el cilindro es de 1 500 psi.

![](./imgs/figure_11_33.png)

## Import necessary dependency

In [1]:
import numpy as np
import pandas as pd
from sympy import Eq, solve, symbols
from utils import *

In [2]:
get_a = lambda j : pow(np.e, 0.0598 * pow(np.log(j), 3) + 0.1385 * pow(np.log(j), 2) - 0.4350 * np.log(j) - 2.3516)

In [3]:
# Datos
Dp = 4 # in
Dbc = 5.5 # in
Df = 7.25 # in


# perno
num_pernos = 8
d = 3/8 # in
d_r = 0.2938 # in
A_t = 0.0775 # in ^ 2

# junta
l_c  = 1.125 # in
l = 1.875 # in
l_a = l - l_c

# sistema
presion = 1500 # psi

t_confinado = 0.125 # in

# Constantes
E_aluminio = 10.4e6 # psi
E_acero =  30e6 # psi
E_caucho = 10e3 #psi


In [4]:
table_11_8 = pd.read_csv('table_11_8.csv')
table_11_8.head()

,j,p0,p1,p2,p3
0,0.1,0.4389,-0.9197,0.8901,-0.3187
1,0.2,0.6118,-1.1715,1.0875,-0.3806
2,0.3,0.6932,-1.2426,1.1177,-0.3845
3,0.4,0.7351,-1.2612,1.1111,-0.3779
4,0.5,0.7580,-1.2632,1.0979,-0.3708


In [5]:
table_11_9 = pd.read_csv('table_11_9.csv')
table_11_9.head()

,j,q0,q1,q2,q3,q4,q5
0,0.1,0.0079,17.0400,-92.832,202.440,-209.38,82.726
1,0.2,0.1010,8.5465,-24.166,15.497,NaN,NaN
2,0.3,0.0861,8.2344,-22.274,13.963,NaN,NaN
3,0.4,0.0695,8.0297,-20.727,12.646,NaN,NaN
4,0.5,0.0533,7.8676,-19.357,11.457,NaN,NaN


## Confinado 

In [6]:
t = get_razon_espesor(l_c, l - l_c)
print(f"Razón  espesor {t}")

j = d / l 
print(f"Razón de aspecto j = {j}")

a = get_a(j)
print(f"factor de corrección a = {a}")

Razón  espesor 0.6
Razón de aspecto j = 0.2
factor de corrección a = 0.21394416695744062


In [7]:
# Hallar la rigidez a cada material
p0, p1, p2, p3, *_ = get_row_values(table_11_8, j)

r_l = E_aluminio / E_acero
print("razón de modulo en material con modulo de young menor", r_l)

c_l = calculate_c_r(r_l, p0, p1, p2, p3)
print("cl", c_l)

r_h = E_acero / E_acero
print("razón de modulo en material con modulo de young mayor", r_h)

c_h = calculate_c_r(r_h, p0, p1, p2, p3)
print("ch", c_h)

# Hallar la rigidez a diferente material
q0, q1, q2, q3, *_ = get_row_values(table_11_9, j)
# print("q0", q0, "q1", q1, "q2", q2, "q3", q3, _)

c_t = calculate_c_t(t, q0, q1, q2, q3, _)
print("ct", c_t)

c = c_h + (t + a * c_t) * (c_l - c_h)
print("rigidez en la junta", c)

razón de modulo en material con modulo de young menor 0.3466666666666667
cl 0.32051691709629626
razón de modulo en material con modulo de young mayor 1.0
ch 0.1471999999999999
ct -0.12350800000000103
rigidez en la junta 0.24661045590082692


In [8]:
area_tapa = np.pi * pow(Dp, 2) / 4
print(f"{area_tapa = }")

f_neta = presion * area_tapa
print(f"{f_neta = }")

p_unitaria = f_neta / num_pernos # psi
print(f"{p_unitaria = }", "lb")

p_b_tornillo = c * p_unitaria
print("carga en el tornillo", p_b_tornillo, "lb")

p_m_material = (1 - c) * p_unitaria
print("carga en el material", p_m_material, "lb")


area_tapa = 12.566370614359172
f_neta = 18849.55592153876
p_unitaria = 2356.194490192345 lb
carga en el tornillo 581.0621974173506 lb
carga en el material 1775.1322927749943 lb


## No Confinado 

In [9]:

l_cuerda = 2 * d + 1/4
print("longitud de la cuerda", l_cuerda)

# Asuma l = 2.5
l_tornillo = 2.5 #in
l_s = l_tornillo - l_cuerda
l_t = l - l_s

print(f"{l_s = }")
print(f"{l_t = }")

area_material_perno = np.pi * pow(d, 2) / 4
print(f"{area_material_perno = }")

k_b_prima = pow(1 + j, -1) * (A_t * area_material_perno) / (area_material_perno * l_t + A_t * l_s) * E_acero
print(f"{k_b_prima}")

longitud de la cuerda 1.0
l_s = 1.5
l_t = 0.375
area_material_perno = 0.11044661672776616
1357225.4607127479


In [10]:
k_m = symbols('k_m')

eqt = Eq(c, k_b_prima/ (k_b_prima+k_m))
k_m = solve(eqt)[0]
print(k_m)

4146294.07074844


In [11]:
area_caucho_plano = np.pi / 4  * (pow(Df, 2) - pow(Dp, 2) - 8 * pow(d, 2)  )
print(area_caucho_plano)

area_g = area_caucho_plano / num_pernos
print(f"{area_g = }")


k_g = area_g * E_caucho / t_confinado
print(k_g, f"{k_g:,}", )

k_m_prima = symbols("k_m_prima")
eqt = Eq(1 / k_m_prima, 1 / k_m + 1 / k_g)
print(f"{k_m:,}", f"{k_g:,}")
k_m_prima = solve(eqt)[0]
print(f"{k_m_prima = }")

c = k_b_prima / (k_b_prima + k_m_prima)
print("Rigidez del material de la junta", c)

p_b_tornillo = c * p_unitaria
print("carga en el tornillo", p_b_tornillo, "lb")

p_m_material = (1 - c) * p_unitaria
print("carga en el material", p_m_material, "lb")


27.832547415397073
area_g = 3.479068426924634
278325.4741539707 278,325.4741539707
4,146,294.07074844 278,325.4741539707
k_m_prima = 260817.738454462
Rigidez del material de la junta 0.838806690335152
carga en el tornillo 1976.39170210416 lb
carga en el material 379.802788088184 lb
